In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('missing').getOrCreate()

In [2]:
# Importing data which has a header and automatically configuring the schema.
df = spark.read.csv('Datasets/forestfires_outliers_removed.csv', header=True, inferSchema=True)
df_missing = spark.read.csv('Datasets/forestfires_missing_outliers_removed.csv', header=True, inferSchema=True)

df_missing.show()
print("Total rows: ", df_missing.count())

+---+---+-----+---+----+----+-----+-----+----+----+----+----+----+
|  X|  Y|month|day|FFMC| DMC|   DC|  ISI|temp|  RH|wind|rain|fire|
+---+---+-----+---+----+----+-----+-----+----+----+----+----+----+
|  0|  7|    5|mar| fri|86.2| 26.2| 94.3| 5.1| 8.2|  51| 6.7| 0.0|
|  1|  7|    4|oct| tue|90.6| 35.4|669.1| 6.7|18.0|  33| 0.9|null|
|  2|  7|    4|oct| sat|90.6| 43.7|686.9| 6.7|14.6|  33| 1.3| 0.0|
|  3|  8|    6|mar| fri|91.7| 33.3| 77.5| 9.0| 8.3|  42| 4.0| 0.2|
|  4|  8|    6|mar| sun|89.3| 51.3|102.2| 9.6|11.4|  42| 1.8| 0.0|
|  5|  8|    6|aug| sun|92.3| 85.3|488.0|14.7|22.2|  29| 5.4| 0.0|
|  6|  8|    6|aug| mon|92.3| 88.9|495.6| 8.5|24.1|  27| 3.1| 0.0|
|  7|  8|    6|aug| mon|91.5|145.4|608.2|10.7| 8.0|  42| 2.2| 0.0|
|  8|  8|    6|sep| tue|91.0|129.5|692.6| 7.0|13.1|  63| 5.4| 0.0|
|  9|  7|    5|sep| sat|92.5| 88.0|698.6| 7.1|22.8|  40| 4.0| 0.0|
| 10|  7|    5|sep| sat|92.5| 88.0|698.6| 7.1|17.8|  51| 7.2| 0.0|
| 11|  7|    5|sep| sat|92.8| 73.2|713.0| 8.4|19.3|  38| 4.0| 

In [3]:
# Remove rows with null values
dropped_df = df_missing.na.drop()
dropped_df.show()
print("Total data points:", dropped_df.count())

+---+---+-----+---+----+----+-----+-----+----+----+----+----+----+
|  X|  Y|month|day|FFMC| DMC|   DC|  ISI|temp|  RH|wind|rain|fire|
+---+---+-----+---+----+----+-----+-----+----+----+----+----+----+
|  0|  7|    5|mar| fri|86.2| 26.2| 94.3| 5.1| 8.2|  51| 6.7| 0.0|
|  2|  7|    4|oct| sat|90.6| 43.7|686.9| 6.7|14.6|  33| 1.3| 0.0|
|  3|  8|    6|mar| fri|91.7| 33.3| 77.5| 9.0| 8.3|  42| 4.0| 0.2|
|  4|  8|    6|mar| sun|89.3| 51.3|102.2| 9.6|11.4|  42| 1.8| 0.0|
|  5|  8|    6|aug| sun|92.3| 85.3|488.0|14.7|22.2|  29| 5.4| 0.0|
|  6|  8|    6|aug| mon|92.3| 88.9|495.6| 8.5|24.1|  27| 3.1| 0.0|
|  7|  8|    6|aug| mon|91.5|145.4|608.2|10.7| 8.0|  42| 2.2| 0.0|
|  8|  8|    6|sep| tue|91.0|129.5|692.6| 7.0|13.1|  63| 5.4| 0.0|
|  9|  7|    5|sep| sat|92.5| 88.0|698.6| 7.1|22.8|  40| 4.0| 0.0|
| 10|  7|    5|sep| sat|92.5| 88.0|698.6| 7.1|17.8|  51| 7.2| 0.0|
| 11|  7|    5|sep| sat|92.8| 73.2|713.0| 8.4|19.3|  38| 4.0| 0.0|
| 12|  6|    5|aug| fri|91.6| 70.8|665.3| 0.8|17.0|  72| 6.7| 

In [4]:
from pyspark.sql.functions import when

# add new fire column to first data set
added_df = df.withColumn('fire', when(df['area'] > 0, 1).otherwise(0))

added_df.show()

+---+---+-----+---+----+----+-----+-----+----+----+----+----+----+----+
|  X|  Y|month|day|FFMC| DMC|   DC|  ISI|temp|  RH|wind|rain|area|fire|
+---+---+-----+---+----+----+-----+-----+----+----+----+----+----+----+
|  0|  7|    5|mar| fri|86.2| 26.2| 94.3| 5.1| 8.2|  51| 6.7| 0.0|   0|
|  1|  7|    4|oct| tue|90.6| 35.4|669.1| 6.7|18.0|  33| 0.9| 0.0|   0|
|  2|  7|    4|oct| sat|90.6| 43.7|686.9| 6.7|14.6|  33| 1.3| 0.0|   0|
|  3|  8|    6|mar| fri|91.7| 33.3| 77.5| 9.0| 8.3|  42| 4.0| 0.2|   1|
|  4|  8|    6|mar| sun|89.3| 51.3|102.2| 9.6|11.4|  42| 1.8| 0.0|   0|
|  5|  8|    6|aug| sun|92.3| 85.3|488.0|14.7|22.2|  29| 5.4| 0.0|   0|
|  6|  8|    6|aug| mon|92.3| 88.9|495.6| 8.5|24.1|  27| 3.1| 0.0|   0|
|  7|  8|    6|aug| mon|91.5|145.4|608.2|10.7| 8.0|  42| 2.2| 0.0|   0|
|  8|  8|    6|sep| tue|91.0|129.5|692.6| 7.0|13.1|  63| 5.4| 0.0|   0|
|  9|  7|    5|sep| sat|92.5| 88.0|698.6| 7.1|22.8|  40| 4.0| 0.0|   0|
| 10|  7|    5|sep| sat|92.5| 88.0|698.6| 7.1|17.8|  51| 7.2| 0.

In [5]:
added_df.describe('fire').show()

+-------+--------------------+
|summary|                fire|
+-------+--------------------+
|  count|                 517|
|   mean|0.015473887814313346|
| stddev|  0.1235474432416998|
|    min|                   0|
|    max|                   1|
+-------+--------------------+



In [6]:
# Drop area column in first added_df
added_df = added_df.drop('area')

# Merge modified datasets
df_merged = dropped_df.union(added_df)
df_merged.show()
print("total rows", df_merged.count())

+---+---+-----+---+----+----+-----+-----+----+----+----+----+----+
|  X|  Y|month|day|FFMC| DMC|   DC|  ISI|temp|  RH|wind|rain|fire|
+---+---+-----+---+----+----+-----+-----+----+----+----+----+----+
|  0|  7|    5|mar| fri|86.2| 26.2| 94.3| 5.1| 8.2|  51| 6.7| 0.0|
|  2|  7|    4|oct| sat|90.6| 43.7|686.9| 6.7|14.6|  33| 1.3| 0.0|
|  3|  8|    6|mar| fri|91.7| 33.3| 77.5| 9.0| 8.3|  42| 4.0| 0.2|
|  4|  8|    6|mar| sun|89.3| 51.3|102.2| 9.6|11.4|  42| 1.8| 0.0|
|  5|  8|    6|aug| sun|92.3| 85.3|488.0|14.7|22.2|  29| 5.4| 0.0|
|  6|  8|    6|aug| mon|92.3| 88.9|495.6| 8.5|24.1|  27| 3.1| 0.0|
|  7|  8|    6|aug| mon|91.5|145.4|608.2|10.7| 8.0|  42| 2.2| 0.0|
|  8|  8|    6|sep| tue|91.0|129.5|692.6| 7.0|13.1|  63| 5.4| 0.0|
|  9|  7|    5|sep| sat|92.5| 88.0|698.6| 7.1|22.8|  40| 4.0| 0.0|
| 10|  7|    5|sep| sat|92.5| 88.0|698.6| 7.1|17.8|  51| 7.2| 0.0|
| 11|  7|    5|sep| sat|92.8| 73.2|713.0| 8.4|19.3|  38| 4.0| 0.0|
| 12|  6|    5|aug| fri|91.6| 70.8|665.3| 0.8|17.0|  72| 6.7| 

In [7]:
# Creating new column with month as numeric values
df_merged = df_merged.withColumn('month nominal', when(df_merged['month'] == 'jan', 1)
                                 .when(df_merged['month'] == 'feb', 2)
                                 .when(df_merged['month'] == 'mar', 3)
                                 .when(df_merged['month'] == 'apr', 4)
                                 .when(df_merged['month'] == 'may', 5)
                                 .when(df_merged['month'] == 'jun', 6)
                                 .when(df_merged['month'] == 'jul', 7)
                                 .when(df_merged['month'] == 'aug', 8)
                                 .when(df_merged['month'] == 'sep', 9)
                                 .when(df_merged['month'] == 'oct', 10)
                                 .when(df_merged['month'] == 'nov', 11)
                                 .when(df_merged['month'] == 'dec', 12)
                                 .otherwise(0))
df_merged.drop('_c0')
df_merged.show()

+---+---+-----+---+----+----+-----+-----+----+----+----+----+----+-------------+
|  X|  Y|month|day|FFMC| DMC|   DC|  ISI|temp|  RH|wind|rain|fire|month nominal|
+---+---+-----+---+----+----+-----+-----+----+----+----+----+----+-------------+
|  0|  7|    5|mar| fri|86.2| 26.2| 94.3| 5.1| 8.2|  51| 6.7| 0.0|            0|
|  2|  7|    4|oct| sat|90.6| 43.7|686.9| 6.7|14.6|  33| 1.3| 0.0|            0|
|  3|  8|    6|mar| fri|91.7| 33.3| 77.5| 9.0| 8.3|  42| 4.0| 0.2|            0|
|  4|  8|    6|mar| sun|89.3| 51.3|102.2| 9.6|11.4|  42| 1.8| 0.0|            0|
|  5|  8|    6|aug| sun|92.3| 85.3|488.0|14.7|22.2|  29| 5.4| 0.0|            0|
|  6|  8|    6|aug| mon|92.3| 88.9|495.6| 8.5|24.1|  27| 3.1| 0.0|            0|
|  7|  8|    6|aug| mon|91.5|145.4|608.2|10.7| 8.0|  42| 2.2| 0.0|            0|
|  8|  8|    6|sep| tue|91.0|129.5|692.6| 7.0|13.1|  63| 5.4| 0.0|            0|
|  9|  7|    5|sep| sat|92.5| 88.0|698.6| 7.1|22.8|  40| 4.0| 0.0|            0|
| 10|  7|    5|sep| sat|92.5

In [8]:
#df_merged.toPandas().to_csv('Datasets/forestfires_merged.csv')